# 📦 IDX Data Collector (GoAPI) — OHLCV + Broker Summary

In [ ]:
# ============================================
# IDX Collector (multi-year) – Fast IO + Audit
# - Parallel HTTP (thread pool) + connection pooling
# - Progress bar per HARI & per SAHAM
# - Resume-friendly (skip jika file sudah ada)
# - Rate-limit & retry w/ backoff
# - Broker optional (N hari terakhir / force all)
# - Lenient snapshot (ret_1/vol_ratio dengan default aman)
# - Audit kelengkapan: missing_YYYY-MM-DD.csv
# ============================================

import os, re, time, random, datetime as dt
from typing import List, Dict, Any, Optional, Tuple
import numpy as np
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **kwargs): return x

# ================== CONFIG ==================
API_BASE_URL = os.getenv("GOAPI_BASE_URL", "https://api.goapi.io")
API_KEY      = os.getenv("GOAPI_API_KEY",   "b217abcc-33e4-5678-4e2f-c858a3c8")
DATA_DIR     = os.getenv("DATA_DIR",        "data")
WATCHLIST    = os.getenv("WATCHLIST",       "nama_saham.csv")  # kolom 'Code' / 'Ticker' / 'Symbol' (flex)

# Histori fitur & horizon backfill
LOOKBACK_DAYS_FOR_FEATURES = int(os.getenv("LOOKBACK_DAYS_FOR_FEATURES", "180"))
YEARS_TO_BACKFILL          = int(os.getenv("YEARS_TO_BACKFILL", "1"))  # mulai 1 tahun

# Kalender (B = business day)
CALENDAR_FREQ = os.getenv("CALENDAR_FREQ", "B")

# HTTP & parallelism
REQ_TIMEOUT_SEC   = float(os.getenv("REQ_TIMEOUT_SEC", "30"))
MAX_RETRY         = int(os.getenv("MAX_RETRY", "3"))
RETRY_BACKOFF_MIN = float(os.getenv("RETRY_BACKOFF_MIN", "0.75"))
RETRY_BACKOFF_MAX = float(os.getenv("RETRY_BACKOFF_MAX", "2.0"))
# batasi jumlah koneksi paralel supaya tidak kena rate limit
MAX_WORKERS       = int(os.getenv("MAX_WORKERS", "16"))
# rate limiter ringan antar request
RATE_LIMIT_SLEEP  = float(os.getenv("RATE_LIMIT_SLEEP", "0.01"))

# Broker summary strategy saat backfill panjang
BROKER_FOR_BACKFILL_DAYS     = int(os.getenv("BROKER_FOR_BACKFILL_DAYS", "5"))   # 5 hari terakhir saja
ENABLE_BROKER_DURING_BACKFILL= os.getenv("ENABLE_BROKER_DURING_BACKFILL", "true").lower() == "true"
FORCE_BROKER_ALL             = os.getenv("FORCE_BROKER_ALL", "false").lower() == "true"

# Bandarmologi
RETAIL_BROKERS = {"XL","YP","CC","PD","NI","AI","MIR"}

os.makedirs(DATA_DIR, exist_ok=True)

# ================== LOGGING ==================
from datetime import datetime
def log(msg: str) -> None:
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{ts}] {msg}", flush=True)

# ================== HTTP (Session Pool) =====
def make_session() -> requests.Session:
    s = requests.Session()
    adapter = requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100)
    s.mount("http://", adapter)
    s.mount("https://", adapter)
    s.headers.update({"User-Agent": "idx-collector/1.0"})
    return s

# thread-local session
from threading import local as _thread_local
_tls = _thread_local()
def get_session() -> requests.Session:
    if not hasattr(_tls, "session"):
        _tls.session = make_session()
    return _tls.session

def http_get(path: str, params: Optional[dict] = None) -> dict:
    url = f"{API_BASE_URL.rstrip('/')}/{path.lstrip('/')}"
    p = dict(params or {})
    p["api_key"] = API_KEY
    last_err = None
    for attempt in range(1, MAX_RETRY + 1):
        try:
            r = get_session().get(url, params=p, timeout=REQ_TIMEOUT_SEC)
            if r.status_code == 200:
                try:
                    return r.json()
                except Exception:
                    raise RuntimeError(f"Non-JSON response: {r.text[:200]}")
            else:
                raise RuntimeError(f"HTTP {r.status_code}: {r.text[:200]}")
        except Exception as e:
            last_err = e
            if attempt < MAX_RETRY:
                time.sleep(random.uniform(RETRY_BACKOFF_MIN, RETRY_BACKOFF_MAX))
            # else: bubble up
        finally:
            if RATE_LIMIT_SLEEP > 0:
                time.sleep(RATE_LIMIT_SLEEP)
    raise last_err if last_err else RuntimeError("Unknown HTTP error")

def _first_list(obj: Any):
    if isinstance(obj, list) and obj and isinstance(obj[0], dict): return obj
    if isinstance(obj, dict):
        for v in obj.values():
            res = _first_list(v);
            if res: return res
    return None

# ================== WATCHLIST ==================
def load_watchlist(path: str) -> List[str]:
    assert os.path.exists(path), f"Watchlist tidak ditemukan: {path}"
    df = pd.read_csv(path, sep=None, engine="python", encoding="utf-8-sig")
    cols_low = {c.lower(): c for c in df.columns}
    candidates = ["code","ticker","symbol","kode","emiten","kode_saham","kode emiten"]
    col = None
    for k in candidates:
        if k in cols_low:
            col = cols_low[k]; break
    if col is None:
        # heuristik: kolom paling mirip ticker
        best, best_ratio = None, -1.0
        for c in df.columns:
            vals = df[c].astype(str).str.strip().str.upper()
            ratio = vals.str.fullmatch(r"[A-Z]{2,5}(\.JK)?").mean()
            if ratio > best_ratio:
                best_ratio, best = ratio, c
        if best_ratio >= 0.6:
            col = best
        else:
            raise ValueError(f"Tidak menemukan kolom ticker. Kolom: {list(df.columns)}")

    syms = (df[col].astype(str)
                .str.strip().str.upper()
                .str.replace(r"\.JK$", "", regex=True))
    syms = syms[syms.str.fullmatch(r"[A-Z]{2,5}")].dropna().unique().tolist()
    syms = sorted(syms)
    log(f"[WATCHLIST] {len(syms)} tickers. Contoh: {syms[:10]}")
    return syms

# ================== FETCHERS ==================
def fetch_history(symbol: str, start: str, end: str) -> pd.DataFrame:
    s = dt.date.fromisoformat(start); e = dt.date.fromisoformat(end)
    step = dt.timedelta(days=360)
    cur = s; rows = []
    while cur <= e:
        chunk_end = min(cur + step, e)
        js = http_get(f"/stock/idx/{symbol}/historical", {"from": cur.isoformat(),"to": chunk_end.isoformat()})
        rows += _first_list(js) or []
        cur = chunk_end + dt.timedelta(days=1)
    if not rows:
        return pd.DataFrame(columns=["timestamp","close","volume"])
    df = pd.DataFrame(rows).copy()
    # normalisasi nama kolom
    lower = {c.lower(): c for c in df.columns}
    def norm(cands, dst):
        for cc in cands:
            if cc in lower:
                df.rename(columns={lower[cc]: dst}, inplace=True)
                return
        if dst not in df.columns: df[dst] = np.nan
    norm(["date","tanggal","time","timestamp"], "timestamp")
    norm(["close","c","last","close_price","price"], "close")
    norm(["volume","v","vol"], "volume")

    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df["close"]     = pd.to_numeric(df["close"],   errors="coerce")
    df["volume"]    = pd.to_numeric(df["volume"],  errors="coerce")

    df = (df.sort_values("timestamp")
            .dropna(subset=["timestamp","close"])
            .drop_duplicates(subset=["timestamp"])
            .reset_index(drop=True))
    return df[["timestamp","close","volume"]]

def fetch_broker_summary(symbol: str, date_iso: str) -> pd.DataFrame:
    js = http_get(f"/stock/idx/{symbol}/broker_summary", {"date": date_iso})
    rows = _first_list(js) or []
    if not rows:
        return pd.DataFrame(columns=["date","symbol","broker","net_value"])
    df = pd.json_normalize(rows, sep="_")
    if "value" not in df.columns or "side" not in df.columns:
        return pd.DataFrame(columns=["date","symbol","broker","net_value"])
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df["net_value"] = np.where(df["side"].astype(str).str.upper()=="BUY", df["value"], -df["value"])
    broker_col = "broker_code" if "broker_code" in df.columns else ("code" if "code" in df.columns else None)
    out = pd.DataFrame({
        "date":   date_iso,
        "symbol": symbol,
        "broker": (df[broker_col].astype(str) if broker_col else None),
        "net_value": df["net_value"]
    })
    return out.dropna(subset=["net_value"], how="all")

def aggregate_broker(bs: pd.DataFrame) -> pd.DataFrame:
    if bs is None or bs.empty:
        return pd.DataFrame(columns=[
            "date","symbol","total_net_value","top_buyer",
            "top_buyer_concentration","top_buyer_net_value",
            "num_buyers","num_sellers","num_brokers","retail_broker_ratio"
        ])
    rows = []
    for (d, sym), sub in bs.groupby(["date","symbol"]):
        sub = sub.copy()
        nv  = pd.to_numeric(sub["net_value"], errors="coerce").fillna(0.0)
        total_buy  = nv[nv > 0].sum()
        pos = sub[nv > 0].copy()
        pos["net_value"] = nv[nv > 0].values
        pos = pos.sort_values("net_value", ascending=False)
        conc = (pos["net_value"].head(2).sum() / total_buy) if total_buy > 0 else 0.0
        rows.append({
            "date": d,
            "symbol": sym,
            "total_net_value": float(nv.sum()),
            "top_buyer": pos["broker"].iloc[0] if len(pos) else None,
            "top_buyer_concentration": float(conc),
            "top_buyer_net_value": float(pos["net_value"].iloc[0]) if len(pos) else 0.0,
            "num_buyers": int((nv > 0).sum()),
            "num_sellers": int((nv < 0).sum()),
            "num_brokers": int(sub["broker"].nunique()),
            "retail_broker_ratio": float(sub["broker"].astype(str).isin(RETAIL_BROKERS).mean()),
        })
    return pd.DataFrame(rows)

# ================== PER-SYMBOL JOB =============
def process_symbol_for_day(sym: str, date: dt.date, start_hist: str, end_hist: str,
                           want_broker: bool) -> Tuple[Optional[dict], Optional[pd.DataFrame], Optional[str]]:
    """
    1) Fetch history (untuk fitur) → ambil baris terakhir
    2) (opsional) Fetch broker summary untuk date
    Return:
      - snapshot row dict (atau None)
      - broker df (atau None)
      - reason kalau gagal (untuk audit)
    """
    try:
        dfh = fetch_history(sym, start_hist, end_hist)
    except Exception as e:
        return None, None, f"http_fail:{e}"
    if dfh is None or dfh.empty:
        return None, None, "empty_history"

    d = dfh.sort_values("timestamp").reset_index(drop=True)
    # ---- fitur aman (lenient) ----
    # ret_1: 0.0 bila tidak ada hari sebelumnya
    if len(d) >= 2:
        ret_1_last = float((d["close"].pct_change()).iloc[-1])
        if not np.isfinite(ret_1_last): ret_1_last = 0.0
    else:
        ret_1_last = 0.0

    vol_ma20_last = float(d["volume"].rolling(20, min_periods=1).mean().iloc[-1] or 0.0)
    denom = max(vol_ma20_last, 1e-9)
    vol_ratio_last = float((d["volume"].iloc[-1] or 0.0) / denom)

    close_last = d["close"].iloc[-1]
    vol_last   = d["volume"].iloc[-1]
    if not np.isfinite(close_last):
        return None, None, "no_close_last"

    snap_row = {
        "date": date,
        "symbol": sym,
        "close": float(close_last),
        "ret_1": float(ret_1_last),
        "vol_ratio": float(vol_ratio_last),
        "volume": float(vol_last if np.isfinite(vol_last) else 0.0),
    }

    bs_df = None
    if want_broker:
        try:
            bs = fetch_broker_summary(sym, date.isoformat())
            if bs is not None and not bs.empty:
                bs_df = bs
        except Exception as e:
            # broker gagal tidak fatal
            bs_df = None

    return snap_row, bs_df, None

# ================== COLLECT PER DAY =============
def collect_day_parallel(date: dt.date, symbols: List[str], want_broker: bool) -> Dict[str, str]:
    start_hist = (date - dt.timedelta(days=LOOKBACK_DAYS_FOR_FEATURES)).isoformat()
    end_hist   = date.isoformat()

    snap_rows: List[Dict[str, Any]] = []
    bs_all: List[pd.DataFrame] = []
    missing: List[Dict[str, str]] = []

    # pool paralel
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futures = [
            ex.submit(process_symbol_for_day, sym, date, start_hist, end_hist, want_broker)
            for sym in symbols
        ]
        for fut in tqdm(as_completed(futures), total=len(futures), desc=f"[{date}] saham", leave=False):
            try:
                snap_row, bs_df, reason = fut.result()
                if snap_row:
                    snap_rows.append(snap_row)
                else:
                    missing.append({"symbol": snap_row["symbol"] if snap_row else "-", "reason": reason or "unknown"})
                if bs_df is not None and not bs_df.empty:
                    bs_all.append(bs_df)
            except Exception as e:
                missing.append({"symbol": "-", "reason": f"future_error:{e}"})

    # simpan snapshot
    snapshot_path = os.path.join(DATA_DIR, f"daily_snapshot_{date}.csv")
    pd.DataFrame(snap_rows).to_csv(snapshot_path, index=False)

    # simpan broker agg bila ada
    broker_agg_path = ""
    if bs_all:
        bs   = pd.concat(bs_all, ignore_index=True)
        agg  = aggregate_broker(bs)
        broker_agg_path = os.path.join(DATA_DIR, f"broker_agg_{date}.csv")
        agg.to_csv(broker_agg_path, index=False)

    # audit missing
    if missing:
        miss_df = pd.DataFrame(missing)
        miss_df["symbol"] = miss_df["symbol"].astype(str)
        miss_df.to_csv(os.path.join(DATA_DIR, f"missing_{date}.csv"), index=False)

    # ringkasan
    log(f"[{date}] snapshot_rows={len(snap_rows)} / target={len(symbols)} | broker_agg={'OK' if broker_agg_path else '-'} | missing={len(missing)}")
    return {"snapshot": snapshot_path, "broker_agg": broker_agg_path}

# ================== ENTRYPOINT ==================
if __name__ == "__main__":
    # 1) load watchlist
    syms = load_watchlist(WATCHLIST)
    log(f"Start COLLECTOR | years={YEARS_TO_BACKFILL} | lookback_days={LOOKBACK_DAYS_FOR_FEATURES} | workers={MAX_WORKERS}")

    # 2) range tanggal (hari bursa)
    today = dt.date.today()
    start_date = today - dt.timedelta(days=YEARS_TO_BACKFILL * 365)
    dates = pd.date_range(start_date, today, freq=CALENDAR_FREQ)

    # 3) progress per-hari
    files_written = 0
    t0 = time.time()
    for d in tqdm(dates, desc="Tanggal (EOD)", unit="day"):
        day = d.date()
        snap_file = os.path.join(DATA_DIR, f"daily_snapshot_{day}.csv")
        if os.path.exists(snap_file):
            # sudah ada—lewati (idempotent)
            continue

        # strategi broker
        delta_days = (today - day).days
        want_broker = FORCE_BROKER_ALL or (ENABLE_BROKER_DURING_BACKFILL and delta_days <= BROKER_FOR_BACKFILL_DAYS)

        try:
            out = collect_day_parallel(day, syms, want_broker=want_broker)
            files_written += 1
            # log detail sudah di dalam collect_day_parallel
        except Exception as e:
            log(f"[ERROR] {day}: {e}")

    dt_s = time.time() - t0
    log(f"SELESAI. Hari diproses: {len(dates)} | snapshot baru: {files_written} | durasi: {dt_s/60:.1f} menit")

[2025-09-02 23:56:59] [WATCHLIST] 954 tickers. Contoh: ['AADI', 'AALI', 'ABBA', 'ABDA', 'ABMM', 'ACES', 'ACRO', 'ACST', 'ADCP', 'ADES']
[2025-09-02 23:56:59] Start COLLECTOR | years=1 | lookback_days=180 | workers=16


Tanggal (EOD):   0%|          | 0/262 [00:00<?, ?day/s]

[2024-09-02] saham:   0%|          | 0/954 [00:00<?, ?it/s]

[2025-09-02 23:58:00] [2024-09-02] snapshot_rows=925 / target=954 | broker_agg=- | missing=29


[2024-09-03] saham:   0%|          | 0/954 [00:00<?, ?it/s]

...

[2025-09-03 03:49:45] [2025-08-29] snapshot_rows=932 / target=954 | broker_agg=OK | missing=22


[2025-09-01] saham:   0%|          | 0/954 [00:00<?, ?it/s]

[2025-09-03 03:51:34] [2025-09-01] snapshot_rows=932 / target=954 | broker_agg=OK | missing=22


[2025-09-02] saham:   0%|          | 0/954 [00:00<?, ?it/s]

[2025-09-03 03:53:15] [2025-09-02] snapshot_rows=932 / target=954 | broker_agg=OK | missing=22
[2025-09-03 03:53:15] SELESAI. Hari diproses: 262 | snapshot baru: 262 | durasi: 236.3 menit


In [ ]:
import os, glob, json, datetime as dt
import pandas as pd, numpy as np, joblib
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

DATA_DIR="data"; MODEL_DIR="models"; os.makedirs(MODEL_DIR,exist_ok=True)

# Load snapshots
snap_files=glob.glob(f"{DATA_DIR}/daily_snapshot_*.csv")
agg_files=glob.glob(f"{DATA_DIR}/broker_agg_*.csv")
snap=pd.concat([pd.read_csv(f,parse_dates=["date"]) for f in snap_files])
agg=pd.concat([pd.read_csv(f,parse_dates=["date"]) for f in agg_files])
df=snap.merge(agg,on=["date","symbol"],how="left")

# Labels
df=df.sort_values(["symbol","date"])
df["next_close"]=df.groupby("symbol")["close"].shift(-1)
df["y_up5"]=((df["next_close"]/df["close"]-1)>=0.05).astype(int)

# Features
df["is_price_lt_500"]=(df["close"]<500).astype(int)
features=["close","ret_1","vol_ratio","volume",
          "top_buyer_concentration","top_buyer_net_value",
          "total_net_value","num_buyers","num_sellers",
          "num_brokers","retail_broker_ratio","is_price_lt_500"]
df=df.dropna(subset=["y_up5"])
X=df[features].fillna(0).values; y=df["y_up5"].values

# Split time-based
split=int(len(df)*0.8)
Xtr,ytr=X[:split],y[:split]; Xte,yte=X[split:],y[split:]

# Train model
try:
    clf=XGBClassifier(n_estimators=500,max_depth=5,learning_rate=0.05,
        subsample=0.9,colsample_bytree=0.9,eval_metric="logloss")
    clf.fit(Xtr,ytr)
except:
    clf=RandomForestClassifier(n_estimators=500,max_depth=10,class_weight="balanced")
    clf.fit(Xtr,ytr)

# Eval
print(classification_report(yte,clf.predict(Xte)))
artifact={"model":clf,"features":features,"target":"y_up5",
          "threshold_default":0.6,"trained_at":dt.datetime.now().isoformat()}
joblib.dump(artifact,f"{MODEL_DIR}/up_model.joblib")

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     45901
           1       0.54      0.07      0.12      2867

    accuracy                           0.94     48768
   macro avg       0.74      0.53      0.55     48768
weighted avg       0.92      0.94      0.92     48768



['models/up_model.joblib']

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, classification_report
import numpy as np

proba = clf.predict_proba(Xte)[:,1] if hasattr(clf,"predict_proba") else clf.decision_function(Xte)

# 1) AUC metrics
try:
    print("ROC-AUC :", roc_auc_score(yte, proba))
    print("PR-AUC  :", average_precision_score(yte, proba))
except Exception as e:
    print("AUC error:", e)

# 2) Cari threshold terbaik utk F1 kelas 1
prec, rec, thr = precision_recall_curve(yte, proba)
f1 = 2*prec*rec/(prec+rec+1e-9)
best_idx = np.nanargmax(f1)
best_thr = thr[max(best_idx-1,0)]  # penyesuaian karena len(thr)=len(prec)-1
print(f"Best-F1 thr={best_thr:.3f}, precision={prec[best_idx]:.3f}, recall={rec[best_idx]:.3f}, f1={f1[best_idx]:.3f}")

# 3) Report di beberapa threshold (mis. 0.3, 0.4, 0.5, best_thr)
for t in [0.3, 0.4, 0.5, float(best_thr)]:
    yhat = (proba >= t).astype(int)
    print(f"\n[thr={t:.2f}]")
    print(classification_report(yte, yhat, digits=4, zero_division=0))

ROC-AUC : 0.7869929867640177
PR-AUC  : 0.2675442577186345
Best-F1 thr=0.201, precision=0.337, recall=0.340, f1=0.339

[thr=0.30]
              precision    recall  f1-score   support

           0     0.9526    0.9796    0.9659     45901
           1     0.4017    0.2194    0.2838      2867

    accuracy                         0.9349     48768
   macro avg     0.6771    0.5995    0.6248     48768
weighted avg     0.9202    0.9349    0.9258     48768


[thr=0.40]
              precision    recall  f1-score   support

           0     0.9485    0.9907    0.9692     45901
           1     0.4831    0.1392    0.2161      2867

    accuracy                         0.9406     48768
   macro avg     0.7158    0.5649    0.5926     48768
weighted avg     0.9212    0.9406    0.9249     48768


[thr=0.50]
              precision    recall  f1-score   support

           0     0.9449    0.9963    0.9699     45901
           1     0.5410    0.0691    0.1225      2867

    accuracy                 

In [ ]:
!zip -r data.zip /content/data

from google.colab import files
files.download("data.zip")

  adding: content/data/ (stored 0%)
  adding: content/data/missing_2025-01-07.csv (deflated 90%)
  adding: content/data/daily_snapshot_2024-12-24.csv (deflated 62%)
  adding: content/data/daily_snapshot_2025-02-06.csv (deflated 62%)
  adding: content/data/daily_snapshot_2025-04-01.csv (deflated 62%)
  adding: content/data/daily_snapshot_2025-08-28.csv (deflated 63%)
  adding: content/data/missing_2025-05-09.csv (deflated 90%)
  adding: content/data/daily_snapshot_2024-10-30.csv (deflated 62%)
  adding: content/data/daily_snapshot_2025-09-01.csv (deflated 63%)
  adding: content/data/missing_2024-09-04.csv (deflated 93%)
  adding: content/data/missing_2024-10-02.csv (deflated 93%)
  adding: content/data/daily_snapshot_2024-09-17.csv (deflated 63%)
  adding: content/data/missing_2025-05-06.csv (deflated 90%)
  adding: content/data/daily_snapshot_2025-01-17.csv (deflated 62%)
  adding: content/data/missing_2025-02-20.csv (deflated 90%)
  adding: content/data/daily_snapshot_2025-08-21.csv (

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>